# 데이콘 Basic 자동차 가격 예측 AI 경진대회

## 대회 정보
- 설명 : 자동차 생산년도, 브랜드, 종류 등 자동차와 관련된 데이터를 활용하여 자동차 가격을 예측하는 AI 모델을 개발
- 규칙 : 외부 데이터 사용 불가 

## 데이터 정보
train.csv
- 57920개의 데이터

test.csv
- 14480개의 데이터

## Column 설명
- ID : 샘플 별 고유 id
- 생산년도 : 차량이 생산된 연도
- 모델출시년도 : 차량의 모델이 처음으로 출시된 연도
- 브랜드
- 차량모델명
- 판매도시 : 3글자로 인코딩된 도시 이름
- 판매구역 : 3글자로 인코딩된 구역 이름
- 주행거리 : 총 주행 거리(km)
- 배기량 : 내연기관에서 피스톤이 최대로 밀어내거나 빨아들이는 부피 (cc)
- 압축천연가스(CNG) : 압축천연가스(CNG) 자동차 여부
- 경유 : 경유 자동차 여부
- 가솔린 : 가솔린 자동차 여부
- 하이브리드 : 하이브리드 자동차 여부
- 액화석유가스(LPG) : 액화석유가스(LPG) 자동차 여부
- 가격 : 자동차 가격(백만원)




## Import

In [17]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc 
rc('font', family='AppleGothic')#맥 한글깨짐

In [18]:
train= pd.read_csv("data/train.csv")
test= pd.read_csv("data/test.csv")

## 데이터 정보

In [19]:
train.head()

,ID,생산년도,모델출시년도,브랜드,차량모델명,판매도시,판매구역,주행거리,배기량,압축천연가스(CNG),경유,가솔린,하이브리드,액화석유가스(LPG),가격
0,TRAIN_00000,2018,2014,skoda,fabia,KAT,SLA,85231,999,0,0,1,0,0,51.74
1,TRAIN_00001,2010,2006,toyota,auris,RKO,SWI,135000,1598,0,0,1,0,0,41.47
2,TRAIN_00002,2002,2002,mercedes-benz,clk-klasa,GNI,WIE,255223,1796,0,0,1,0,0,17.81
3,TRAIN_00003,2006,2001,nissan,x-trail,EHX,WIE,238000,2184,0,1,0,0,0,18.20
4,TRAIN_00004,2007,2007,fiat,bravo,OSW,MAL,251000,1910,0,1,0,0,0,17.55


In [20]:
test.head()

,ID,생산년도,모델출시년도,브랜드,차량모델명,판매도시,판매구역,주행거리,배기량,압축천연가스(CNG),경유,가솔린,하이브리드,액화석유가스(LPG)
0,TEST_00000,2017,2013,mazda,3,KAL,WIE,57000,1998,0,0,1,0,0
1,TEST_00001,2011,2008,ford,fiesta,ROP,POD,159000,1242,0,0,1,0,0
2,TEST_00002,2019,2017,volkswagen,polo,LOD,LOD,11284,999,0,0,1,0,0
3,TEST_00003,2021,2016,renault,megane,JAW,SLA,9,1332,0,0,1,0,0
4,TEST_00004,2010,2011,volvo,v60,WEJ,POM,270000,1999,0,0,1,0,0


In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57920 entries, 0 to 57919
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           57920 non-null  object 
 1   생산년도         57920 non-null  int64  
 2   모델출시년도       57920 non-null  int64  
 3   브랜드          57920 non-null  object 
 4   차량모델명        57920 non-null  object 
 5   판매도시         57920 non-null  object 
 6   판매구역         57920 non-null  object 
 7   주행거리         57920 non-null  int64  
 8   배기량          57920 non-null  int64  
 9   압축천연가스(CNG)  57920 non-null  int64  
 10  경유           57920 non-null  int64  
 11  가솔린          57920 non-null  int64  
 12  하이브리드        57920 non-null  int64  
 13  액화석유가스(LPG)  57920 non-null  int64  
 14  가격           57920 non-null  float64
dtypes: float64(1), int64(9), object(5)
memory usage: 6.6+ MB


In [13]:
print("train : ", train.shape)
print("test : ", test.shape)


train :  (57920, 15)
test :  (14480, 14)


In [23]:
train.isnull().sum()


ID             0
생산년도           0
모델출시년도         0
브랜드            0
차량모델명          0
판매도시           0
판매구역           0
주행거리           0
배기량            0
압축천연가스(CNG)    0
경유             0
가솔린            0
하이브리드          0
액화석유가스(LPG)    0
가격             0
dtype: int64

In [22]:
#고유값
train.nunique()

ID             57920
생산년도              39
모델출시년도            34
브랜드               20
차량모델명            143
판매도시            3224
판매구역              17
주행거리           20994
배기량              347
압축천연가스(CNG)        2
경유                 2
가솔린                2
하이브리드              2
액화석유가스(LPG)        2
가격              2466
dtype: int64

In [25]:
print('train 생산년도 시작년도 : ',np.min(train['생산년도'].values))
print('train 생산년도 마지막년도 : ',np.max(train['생산년도'].values))

train 생산년도 시작년도 :  1978
train 생산년도 마지막년도 :  2022


In [26]:
print('train 모델출시년도 시작년도 : ',np.min(train['모델출시년도'].values))
print('train 모델출시년도 마지막년도 : ',np.max(train['모델출시년도'].values))

train 모델출시년도 시작년도 :  1983
train 모델출시년도 마지막년도 :  2021


In [30]:
train.loc[(train['모델출시년도'] > train['생산년도'] )]
# 이거 진짜 이상하네 

,ID,생산년도,모델출시년도,브랜드,차량모델명,판매도시,판매구역,주행거리,배기량,압축천연가스(CNG),경유,가솔린,하이브리드,액화석유가스(LPG),가격
1048,TRAIN_01048,2015,2016,renault,megane,ROG,SLA,111000,1618,0,0,1,0,0,80.34
1175,TRAIN_01175,2007,2014,ford,mondeo,ZYW,OPO,256000,1997,0,1,0,0,0,20.02
1366,TRAIN_01366,2006,2015,ford,galaxy,PIA,MAZ,193727,1997,0,1,0,0,0,22.10
1857,TRAIN_01857,2005,2007,mitsubishi,lancer,GDA,POM,150000,1997,0,0,1,0,0,5.46
2369,TRAIN_02369,2009,2010,ford,focus,BAN,POM,192000,1596,0,0,1,0,0,24.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56816,TRAIN_56816,2015,2017,volvo,xc-60,TOR,KUJ,134884,2400,0,1,0,0,0,142.87
57028,TRAIN_57028,1992,1993,mercedes-benz,e-klasa,BYL,WIE,69309,4196,0,0,1,0,0,139.10
57357,TRAIN_57357,2007,2008,honda,jazz,KRA,MAL,151997,1246,0,0,1,0,0,20.80
57445,TRAIN_57445,2008,2009,mercedes-benz,e-klasa,OLS,WAR,220100,2143,0,1,0,0,0,60.97
